In [1]:
# import library
import tensorflow as tf
import numpy as np
import os
import time

In [2]:
# download dataset
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 1s 1us/step


In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


In [6]:
# memisahkan karakter-karakter dalam daftar teks menggunakan tf.strings.unicode_split. 
# mengambil daftar teks sebagai input dan memisahkannya menjadi karakter-karakter Unicode.
example_texts = ['abcdefg', 'xyz']
chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [7]:
# membuat sebuah lapisan StringLookup menggunakan TensorFlow. 
# Lapisan ini digunakan untuk mengonversi karakter-karakter dalam teks menjadi ID numerik berdasarkan vocabulary yang diberikan. 
ids_from_chars = tf.keras.layers.StringLookup(vocabulary=list(vocab), mask_token=None)

In [8]:
# menggunakan lapisan StringLookup (ids_from_chars) untuk mengonversi karakter-karakter dalam teks (yang telah dipecah menjadi karakter-karakter Unicode) 
# menjadi ID numerik berdasarkan vocabulary yang telah Anda sediakan.
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [9]:
# membuat lapisan StringLookup lain menggunakan TensorFlow. 
# Lapisan ini digunakan untuk mengonversi ID numerik kembali menjadi karakter-karakter teks asli berdasarkan vocabulary yang telah Anda definisikan sebelumnya. 
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [10]:
#menggunakan lapisan chars_from_ids untuk mengonversi ID numerik (dalam tensor ids) kembali ke karakter-karakter teks. 
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [11]:
# menggunakan TensorFlow (tf.strings.reduce_join) untuk menggabungkan karakter-karakter dalam tensor chars kembali menjadi teks tunggal.
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [12]:
# mengambil ID numerik dan mengonversinya kembali menjadi teks menggunakan lapisan chars_from_ids 
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [13]:
# menggunakan lapisan ids_from_chars untuk mengonversi karakter-karakter dalam teks yang dipecah menggunakan tf.strings.unicode_split menjadi ID numerik.
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [14]:
# membuat sebuah objek tf.data.Dataset menggunakan tf.data.Dataset.from_tensor_slices dari tensor all_ids.
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [15]:
# menggunakan dataset ids_dataset untuk mengambil 10 elemen pertama dari dataset dan kemudian mengonversi ID numerik kembali ke teks menggunakan chars_from_ids. 
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [16]:
# definisi variabel
seq_length = 100

In [17]:
# menggunakan dataset ids_dataset yang telah dibuat sebelumnya dan menggabungkannya menjadi urutan (sequence) dengan panjang sebesar seq_length + 1.
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

# menggunakan loop for untuk mengambil 1 urutan pertama (batch) dari dataset sequences.
for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [18]:
# menggunakan dataset sequences yang telah dibuat sebelumnya untuk mengambil 5 urutan pertama dari dataset. 
# menggunakan fungsi text_from_ids untuk mengonversi ID numerik kembali menjadi teks dari urutan tersebut, dan hasilnya dicetak ke layar.
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [19]:
# mengambil sebuah urutan (sequence) sebagai input dan menghasilkan dua keluaran, yaitu teks input dan teks target.
def split_input_target(sequence):
    # mengambil semua elemen dalam urutan kecuali elemen terakhir. 
    input_text = sequence[:-1]
    # mengambil semua elemen dalam urutan kecuali elemen pertama. 
    target_text = sequence[1:]
    return input_text, target_text

In [20]:
# memisahkan teks input dan teks target dari urutan karakter "Tensorflow".
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [21]:
# menggunakan metode map pada dataset sequences untuk menerapkan fungsi split_input_target pada setiap elemen dalam dataset sequences.
# memisahkan teks input dan teks target dari setiap urutan dalam dataset. 
dataset = sequences.map(split_input_target)

In [22]:
# menggunakan dataset dataset yang telah dibuat sebelumnya untuk mengambil satu pasang contoh (teks input dan teks target) pertama dari dataset.
# menggunakan fungsi text_from_ids untuk mengonversi ID numerik kembali menjadi teks dan mencetaknya ke layar.
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [23]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    # menggunakan metode shuffle untuk mengacak urutan elemen dalam dataset.
    .shuffle(BUFFER_SIZE)
    # menggunakan metode batch untuk mengelompokkan elemen-elemen dataset menjadi batch dengan ukuran sebesar BATCH_SIZE. 
    .batch(BATCH_SIZE, drop_remainder=True)
    # menggunakan metode prefetch untuk mengoptimalkan performa pelatihan model.
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [24]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [25]:
# membuat model
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    # mengkonversi ID numerik menjadi vektor embedding
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    # membuat lapisan GRU (Gated Recurrent Unit) menggunakan tf.keras.layers.GRU. Lapisan ini digunakan untuk model rekurensi.
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    # menghasilkan output yang memiliki dimensi sesuai dengan jumlah kata atau karakter dalam vocabulary.
    self.dense = tf.keras.layers.Dense(vocab_size)

  # mendefinisikan perilaku model saat dijalankan
  #  mengatur bagaimana input akan melalui lapisan-lapisan model.
  # melakukan embedding, menjalankan lapisan GRU, dan menghasilkan output.
  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [26]:
model = MyModel(
    # Jumlah total kata atau karakter dalam vocabulary yang digunakan oleh model.
    vocab_size=vocab_size,
    # Dimensi vektor embedding yang akan digunakan oleh lapisan embedding dalam model.
    embedding_dim=embedding_dim,
    # Jumlah unit dalam lapisan GRU (Gated Recurrent Unit) dalam model.
    rnn_units=rnn_units)

In [27]:
# menggunakan dataset dataset yang telah dipersiapkan sebelumnya untuk mengambil satu batch (batch pertama) dari data latihan.
# menjalankan model model pada input_example_batch untuk mendapatkan prediksi model untuk batch tersebut.
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [28]:
# melihat ringkasan dari arsitektur model, termasuk jumlah parameter dan struktur lapisan.
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4022850 (15.35 MB)
Trainable params: 4022850 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [29]:
# menggunakan tf.random.categorical untuk mengambil sampel acak dari prediksi model yang pertama dalam batch (indeks 0). 
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
# menggunakan tf.squeeze untuk menghilangkan dimensi yang tidak diperlukan (misalnya, menghilangkan dimensi 1 yang dihasilkan oleh tf.random.categorical)
# menggunakan .numpy() untuk mengonversi hasilnya ke dalam format numpy array.
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [30]:
# array yang berisi indeks-indeks kata atau karakter yang telah diambil sebagai sampel acak dari prediksi model
sampled_indices

array([53, 19,  1, 60, 49, 30, 36, 65, 57, 39, 59,  8, 58, 12, 65, 60, 59,
       45,  1, 52, 36, 29, 10, 46, 56, 65, 47,  7,  1,  7,  0, 43,  8, 22,
       15, 46, 32, 42, 43,  4, 43, 56,  3, 25, 30, 52,  1, 51, 13, 38, 50,
       56, 43,  7,  0, 18, 32, 42, 18, 47, 20, 30,  4,  3, 17, 25, 26, 54,
       17, 59, 22,  5,  7, 21, 38, 38, 31, 53, 10,  7, 19,  5, 31, 59, 34,
       12,  0,  7, 48, 26,  5, 36, 20, 31, 36, 39, 58, 15, 26, 56])

In [31]:
# mencetak teks input dari batch pertama dalam dataset. 
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
# mencetak prediksi teks berikutnya yang dihasilkan oleh model
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b"some strong purpose, steel'd\nThe hearts of men, they must perforce have melted\nAnd barbarism itself "

Next Char Predictions:
 b'nF\nujQWzrZt-s;zutf\nmWP3gqzh,\n,[UNK]d-IBgScd$dq!LQm\nl?Ykqd,[UNK]EScEhGQ$!DLMoDtI&,HYYRn3,F&RtU;[UNK],iM&WGRWZsBMq'


In [32]:
# menggunakan fungsi kerugian SparseCategoricalCrossentropy dari TensorFlow. 
# Ini adalah salah satu fungsi kerugian yang umum digunakan dalam tugas klasifikasi, terutama saat bekerja dengan data yang tidak dalam format one-hot encoding.
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [33]:
# menggunakan fungsi kerugian loss yang telah didefinisikan sebelumnya untuk menghitung kerugian model pada batch contoh yang diberikan.
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.190623, shape=(), dtype=float32)


In [34]:
# perhitungan untuk menghitung eksponensial (nilai e pangkat x) dari nilai rata-rata kerugian yang telah dihitung sebelumnya.
tf.exp(example_batch_mean_loss).numpy()

66.06393

In [35]:
# menggunakan optimizer Adam sebagai optimizer yang akan digunakan dalam pelatihan model.
# mengatur fungsi kerugian yang telah Anda definisikan sebelumnya (dalam hal ini, loss) sebagai fungsi kerugian yang akan digunakan selama pelatihan model.
model.compile(optimizer='adam', loss=loss)

In [36]:
# menyiapkan konfigurasi untuk menyimpan checkpoint model selama proses pelatihan.
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

# menyimpan checkpoint model selama pelatihan, sesuai dengan konfigurasi yang telah diatur sebelumnya.
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [37]:
# melatih model Anda selama 20 epoch
EPOCHS = 20

In [38]:
# model Anda akan melalui 20 epoch pelatihan, dan pada setiap akhir epoch, checkpoint model akan disimpan di direktori training_checkpoints.
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
172/172 [==============================] - 21s 61ms/step - loss: 2.7375
Epoch 2/20
172/172 [==============================] - 12s 52ms/step - loss: 2.0029
Epoch 3/20
172/172 [==============================] - 11s 53ms/step - loss: 1.7219
Epoch 4/20
172/172 [==============================] - 11s 53ms/step - loss: 1.5587
Epoch 5/20
172/172 [==============================] - 11s 54ms/step - loss: 1.4574
Epoch 6/20
172/172 [==============================] - 11s 55ms/step - loss: 1.3894
Epoch 7/20
172/172 [==============================] - 12s 55ms/step - loss: 1.3353
Epoch 8/20
172/172 [==============================] - 12s 56ms/step - loss: 1.2918
Epoch 9/20
172/172 [==============================] - 13s 57ms/step - loss: 1.2510
Epoch 10/20
172/172 [==============================] - 13s 58ms/step - loss: 1.2120
Epoch 11/20
172/172 [==============================] - 13s 59ms/step - loss: 1.1731
Epoch 12/20
172/172 [==============================] - 13s 61ms/step - loss: 1.1324
E

In [39]:
# melakukan generasi teks satu langkah pada suatu waktu (one-step generation) dengan model pemodelan bahasa.
class OneStep(tf.keras.Model):
  # metode konstruktor yang digunakan untuk inisialisasi objek OneStep.
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [40]:
# melakukan generasi teks satu langkah pada suatu waktu (one-step generation).
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [41]:
# mengukur waktu mulai proses generasi teks.
start = time.time()
# mengindikasikan bahwa tidak ada state model sebelumnya yang digunakan.
states = None
# eks awal 'ROMEO:' diinisialisasi sebagai tensor konstanta next_char.
next_char = tf.constant(['ROMEO:'])
result = [next_char]

# mengulang sebanyak 1000 karakter untuk generasi teks.
for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

# menggabungkan semua karakter yang dihasilkan menjadi satu teks.
result = tf.strings.join(result)
# mengukur waktu akhir proses generasi teks.
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
Thou dost told me in madam. A stuff it off,
Having more free forth her with the right, and sleep
That hap more man haz but in the flint;
Hust to be seld-men'd in all night,
And each our body summer fail, myself alone;
Her business to be set mock'd now to depose
Ye: or the very need of men.
But curlina scrutch, my lord, thus I'll come you of mine.

KING RICHARD III:
Then call it. Do there be the father and to thanks for from him: if
Thou speak'st, and staptly, pardon alive.

DUCHESS OF YORK:
Axity, Somerset, contemn'd to Henry,
But they have letters much: but sue
The trumpets, and opposed in Christian seal,
Have caused, it will be more than seen his tears:
Which torment me, be but unquiet wrench,
And throw'd murderer work, deflect I have some leggar;
Take ond the accuser at friend,
Have so defear'd that you are happied inform any further
Where some sent o' the all encounter champ,
Think in our brother in Bowit; for my seat contended home,
One rest man's ventulive rover so so done

In [42]:
start = time.time()
states = None
# menginisialisasi lima teks awal 'ROMEO:' sebagai tensor konstanta next_char.
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  # menghasilkan karakter berikutnya berdasarkan next_char dan state model sebelumnya, dan kemudian memperbarui state model.
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

# menggabungkan semua karakter yang dihasilkan menjadi satu teks panjang.
result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO:\nThat hath been still'd my behalf.\n\nLEONTES:\nO,\nsee, the man! why, madame! how shall you be so? And\nWas't moved him your face?\nOr, rough by babes? O woful good deny!\n\nTRANIO:\nGo bear the law usurpin; and therefore hear our maideninable,\nAnd lip is good on thee.\nHere, sirrah, let's follow to the Duke of Norfolk;\nAnd strains of earnestry. O citizens comes troud me?\n\nThird Servant:\nUp the man for this force. But, let's be not a\nMarcius, Selford, bold in person, noble\nMaligious awake, an argosy-foil\nHath anger best in with a flower-on's womb,\nWhile we have shed with Gloucestershire:\nThen wast nothing out for that, my soul! come on,\nAnd from the lead on her slave, rotally at safe\ngenerally agoin, nothing but so:\nThe noble would have springs my mother doth more\nsafely, with all speed, so much blood and bound.\nAnd this, poor seat, swame, I see Queen Margaret say so,\nThat makes these valour as myself.\nAnd single sir, your occunation I may, when\n

In [43]:
# menggunakan tf.saved_model.save untuk menyimpan model one_step_model ke dalam direktori 'one_step'.
tf.saved_model.save(one_step_model, 'one_step')
# memuat kembali model yang telah disimpan dengan menggunakan tf.saved_model.load. 
one_step_reloaded = tf.saved_model.load('one_step')

In [44]:
# menggunakan model yang telah dimuat kembali, yaitu one_step_reloaded, 
# untuk melakukan generasi teks dengan teks awal 'ROMEO:'.
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:
Good morrow, next way hath mine own;
Whilst we shall not in the consulves, myself am all;
Doth so m


TUGAS
---

In [45]:
# mendefinisikan kelas CustomTraining yang merupakan turunan dari model MyModel.
class CustomTraining(MyModel):
   # mengompilasi metode train_step menjadi sebuah graph TensorFlow, meningkatkan kinerja dan efisiensi dalam pelatihan model.
  @tf.function
  # melatih model selama satu langkah pelatihan. 
  # Metode ini menerima input dalam bentuk pasangan (inputs, labels).
  def train_step(self, inputs):
   # mendekomposisi input menjadi inputs dan labels, yang mengandung data input dan label yang sesuai
      inputs, labels = inputs
      # menggunakan tf.GradientTape untuk melacak perhitungan dalam metode ini untuk menghitung gradien loss terhadap parameter-model.
      with tf.GradientTape() as tape:
         # menghitung prediksi model dengan memanggil metode self(inputs, training=True). 
         predictions = self(inputs, training=True)
         # menghitung loss model dengan memanggil metode self.loss dengan labels dan predictions sebagai argumen.
         loss = self.loss(labels, predictions)
      # menghitung gradien loss terhadap parameter-model dengan menggunakan tape yang telah DIbuat sebelumnya.
      grads = tape.gradient(loss, model.trainable_variables)
      # menerapkan gradien ke parameter-model menggunakan optimizer yang telah DIkonfigurasikan sebelumnya.
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))
      # mengembalikan dictionary yang berisi loss sebagai metrik yang dapat digunakan untuk memantau performa pelatihan.
      return {'loss': loss}

In [46]:
# Model ini akan digunakan untuk pelatihan dengan logika pelatihan yang telah DIdefinisikan dalam metode train_step yang khusus. 
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [47]:
# mengonfigurasi model untuk pelatihan dengan menggunakan optimizer Adam dan loss function Sparse Categorical Crossentropy.
model.compile(optimizer = tf.keras.optimizers.Adam(),
             loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [48]:
# memanggil metode fit pada model Anda untuk melatih model dengan dataset yang DIberikan selama 1 epoch.
model.fit(dataset, epochs=1)

172/172 [==============================] - 15s 60ms/step - loss: 2.7188


In [49]:
EPOCHS = 10

# digunakan untuk menghitung rata-rata loss selama pelatihan.
mean = tf.metrics.Mean()

# melakukan loop sebanyak EPOCHS (dalam hal ini, 10 epoch).
for epoch in range(EPOCHS):
    start = time.time()

    # mengatur ulang state rata-rata loss dengan mean.reset_states().
    mean.reset_states()
    # memanggil model.train_step([inp, target]) untuk melatih model dengan satu batch data. 
    # mengupdate rata-rata loss dengan memanggil mean.update_state(logs['loss']).
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        # setiap 50 batch, mencetak informasi seperti nomor epoch, nomor batch, dan loss batch yang dihitung
        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
       model.save_weights(checkpoint_prefix.format(epoch=epoch))

    # mencetak rata-rata loss selama epoch tersebut, waktu yang diperlukan untuk satu epoch, dan garis pembatas.
    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

# Setelah selesai semua epoch, menyimpan bobot model untuk model terlatih terakhir
model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 2.1877
Epoch 1 Batch 50 Loss 2.0517
Epoch 1 Batch 100 Loss 1.9606
Epoch 1 Batch 150 Loss 1.8909

Epoch 1 Loss: 1.9897
Time taken for 1 epoch 14.36 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 1.8155
Epoch 2 Batch 50 Loss 1.7352
Epoch 2 Batch 100 Loss 1.7297
Epoch 2 Batch 150 Loss 1.7112

Epoch 2 Loss: 1.7142
Time taken for 1 epoch 13.31 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 1.5950
Epoch 3 Batch 50 Loss 1.5869
Epoch 3 Batch 100 Loss 1.5605
Epoch 3 Batch 150 Loss 1.5520

Epoch 3 Loss: 1.5541
Time taken for 1 epoch 12.94 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 1.4386
Epoch 4 Batch 50 Loss 1.4714
Epoch 4 Batch 100 Loss 1.4628
Epoch 4 Batch 150 Loss 1.4576

Epoch 4 Loss: 1.4551
Time taken for 1 epoch 12.51 sec
_____________________________________________________________________